In [1]:
import numpy as np
import pandas as pd
# import project_env as pe
import requests
import logging
import time
# import googlemaps # Make sure to install: $ pip install -U googlemaps 
from datetime import timedelta, datetime
import os

In [2]:
# Show all columns when displaying datasets in notebook
from IPython.display import display
pd.options.display.max_columns = None

## Read and merge source files

Here we first read the source files and merge them.

In [3]:
path = "noda_cleaned"
os.listdir(path)

['Ada.csv',
 'Areg.csv',
 'Bdsr.csv',
 'Bond.csv',
 'Caze.csv',
 'Cgcd.csv',
 'Chrg.csv',
 'Dfdn.csv',
 'Disp.csv',
 'Dsum.csv',
 'Event.csv',
 'Judge.csv',
 'Parm.csv',
 'Prcn.csv',
 'Sent.csv',
 'Wdxr.csv']

In [4]:
csv_ls = os.listdir(path)
dfnames = [csv.split('.')[0] for csv in csv_ls]
df_dict = dict.fromkeys(dfnames, None)

for csv in csv_ls:
    df = pd.read_csv(os.path.join(path, csv), index_col=False)
    df_name = csv.split('.')[0]
    df_dict[df_name] = df

C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\sujeong\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9,14,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, ce

### Modify Bond & Chrg & Wdxr Table

In [5]:
# Choose the latest bond for each charge
# AREG_SEQ_NBR was 1 for all records, meaning there were only 1 arrest for each 'sys_nbr - bofi_nbr' combination
idx = df_dict['Bond'].groupby(['SYS_NBR', 'BOFI_NBR', 'CHRG_SEQ_NBR'])['BOND_SEQ_NBR'].transform(max) == df_dict['Bond']['BOND_SEQ_NBR']
df_dict['Bond'] = df_dict['Bond'][idx]
df_dict['Bond']

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,BOND_SEQ_NBR,BOFI_NBR,SUB_CHARGE_CODE,BOND_STATUS_CODE,BOND_TYPE_CODE
0,84000020,1,1,1,1,255544,NaN,JD,RRJ
1,84000133,1,1,1,1,257683,NaN,RM,CMM
2,84000405,1,1,1,1,197485,NaN,RM,CMM
3,84000429,1,1,1,1,142503,DFM,NaN,CMM
4,84000603,1,1,1,1,258357,NaN,JD,RRJ
...,...,...,...,...,...,...,...,...,...
49662,99303723,1,1,3,1,310721,NaN,NaN,PSB
49663,99300543,1,1,2,1,388334,NaN,NaN,CMM
49664,99296442,2,1,1,1,369535,NaN,NaN,CMM
49666,99296442,1,1,2,2,361655,A04P,NaN,RRJ


In [6]:
# Choose the lead charge for each 'sys_nbr - bofi_nbr' pair
# Some pairs have LEAD_CHARGE_FLAG = N for all charges. In this case, we choose the first row by .first()
df_dict['Chrg'] = df_dict['Chrg'].sort_values('LEAD_CHARGE_FLAG', ascending=False).groupby(['SYS_NBR', 'BOFI_NBR']).first()
df_dict['Chrg'].columns = ['DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'CHARGE_DATE', 'LEAD_CHARGE_FLAG', \
                           'SUB_CHARGE_CODE', 'COUNTS', 'CHRG_ADA_CODE', 'CHRG_JUDGE_CODE', 'BOND_MADE_AMOUNT', \
                           'BOND_SET_AMOUNT', 'BOND_MADE_FLAG']

In [7]:
# Filter out WITN_IS_VICTIM_FLAG == N
df_dict['Wdxr'] = df_dict['Wdxr'][df_dict['Wdxr']['WITN_IS_VICTIM_FLAG']=='Y'].drop(columns=['WITN_PTR'])
# Ignore witness type (for now)
df_dict['Wdxr'] = df_dict['Wdxr'].groupby(['SYS_NBR', 'BOFI_NBR']).first()

### Start Merging

In [8]:
######################################
# Merge all data into one data frame #
######################################

# Remove duplicates from Dfdn
df_dict['Dfdn'] = df_dict['Dfdn'].groupby(['BOFI_NBR']).first()
# Merge SENT and DFDN
data_merged = pd.merge(df_dict['Sent'], df_dict['Dfdn'], on='BOFI_NBR', how='left')
data_merged.drop(columns=['CCN_NBR', 'ZIP_DFDN'], inplace=True)
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK
130732,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO
130733,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111.0,5.0,2.0,115.0,F,N,DBR,BRO,BLK
130734,99297292,1,1,3,1,1,282197,40:967 (C) (2),19991116,50,BPQ,N,1,0,0,CNR,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,P6260,PAR,N,Y,635316FA9,19690130.0,5.0,9.0,140.0,M,N,DBR,BRO,BLK


In [9]:
data_merged = pd.merge(data_merged, df_dict['Caze'], on='SYS_NBR', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN
130732,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,NaN
130733,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111.0,5.0,2.0,115.0,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN
130734,99297292,1,1,3,1,1,282197,40:967 (C) (2),19991116,50,BPQ,N,1,0,0,CNR,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,P6260,PAR,N,Y,635316FA9,19690130.0,5.0,9.0,140.0,M,N,DBR,BRO,BLK,NSP,NaN,3.0,1,408206,N,40:967 (C) (2),NaN,NaN


In [10]:
data_merged = pd.merge(data_merged, df_dict['Chrg'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N
130732,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,NaN,19990505.0,Y,NaN,1.0,ALOG,NMB,0.0,0.0,N
130733,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111.0,5.0,2.0,115.0,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,ADAI,NaN,0.0,0.0,N
130734,99297292,1,1,3,1,1,282197,40:967 (C) (2),19991116,50,BPQ,N,1,0,0,CNR,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,P6260,PAR,N,Y,635316FA9,19690130.0,5.0,9.0,140.0,M,N,DBR,BRO,BLK,NSP,NaN,3.0,1,408206,N,40:967 (C) (2),NaN,NaN,19990713.0,Y,A04P,1.0,MABR,PAR,0.0,0.0,N


In [11]:
data_merged = pd.merge(data_merged, df_dict['Cgcd'], left_on='CHARGE_CODE_SENT', right_on='CHARGE_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3
130732,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,NaN,19990505.0,Y,NaN,1.0,ALOG,NMB,0.0,0.0,N,14:67.10 (B),NaN,3
130733,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111.0,5.0,2.0,115.0,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,ADAI,NaN,0.0,0.0,N,NaN,NaN,NaN
130734,99297292,1,1,3,1,1,282197,40:967 (C) (2),19991116,50,BPQ,N,1,0,0,CNR,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,P6260,PAR,N,Y,635316FA9,19690130.0,5.0,9.0,140.0,M,N,DBR,BRO,BLK,NSP,NaN,3.0,1,408206,N,40:967 (C) (2),NaN,NaN,19990713.0,Y,A04P,1.0,MABR,PAR,0.0,0.0,N,40:967 (C) (2),NAR,3


In [12]:
data_merged = pd.merge(data_merged, df_dict['Disp'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'DISP_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130731,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),A40P,19991116.0,50.0,DFM,NaN
130732,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,NaN,19990505.0,Y,NaN,1.0,ALOG,NMB,0.0,0.0,N,14:67.10 (B),NaN,3,14:67.10 (B),NaN,19991116.0,50.0,DFM,NaN
130733,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,NaN,Y,Y,NaN,19841111.0,5.0,2.0,115.0,F,N,DBR,BRO,BLK,JGN,NaN,3.0,1,9931426Q,N,14:68.4,NaN,NaN,19991109.0,Y,NaN,1.0,ADAI,NaN,0.0,0.0,N,NaN,NaN,NaN,14:68.4,NaN,19991115.0,50.0,JFAH,NaN
130734,99297292,1,1,3,1,1,282197,40:967 (C) (2),19991116,50,BPQ,N,1,0,0,CNR,4,0,0,0,0.0,0.0,0.0,0.0,0,N

In [13]:
# One sentencing can have multiple conditions -> thus the number or rows increased
data_merged = pd.merge(data_merged, df_dict['Prcn'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                         'DISP_SEQ_NBR', 'SENT_SEQ_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132410,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),A40P,19991116.0,50.0,DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132411,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,NaN,19990505.0,Y,NaN,1.0,ALOG,NMB,0.0,0.0,N,14:67.10 (B),NaN,3,14:67.10 (B),NaN,19991116.0,50.0,DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132412,99308953,1,1,2,1,1,J115276,15:08.4,19991115,50,JFAH,N,0,6,0,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,DOC,N,U,P2000,Na

In [14]:
data_merged = pd.merge(data_merged, df_dict['Bond'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR',
                                                         'BOFI_NBR', 'SUB_CHARGE_CODE'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132410,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),A40P,19991116.0,50.0,DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132411,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,NaN,19990505.0,Y,NaN,1.0,ALOG,NMB,0.0,0.0,N,14:67.10 (B),NaN,3,14:67.10 (B),NaN,19991116.0,50.0,DFM,NaN,NaN,NaN,NaN,NaN,Na

In [15]:
# multiple bond surety agency can be involced in one bond -> the number of rows increased
data_merged = pd.merge(data_merged, df_dict['Bdsr'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 'BOND_SEQ_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132853,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),A40P,19991116.0,50.0,DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132854,99291741,1,1,3,1,1,214282,14:67.10 (B),19991116,50,DFM,N,1,0,0,SIN,4,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,Y,DOC,N,NaN,B3000,PAR,N,Y,229023V11,19611031.0,5.0,8.0,170.0,M,N,DBR,BRO,BRO,GEN,NaN,3.0,1,406789,N,14:67.10 (B),NaN,N

In [16]:
data_merged = pd.merge(data_merged, df_dict['Judge'], left_on='JUDGE_CODE_SENT', right_on='JUDGE_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132853,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),A40P,19991116.0,50.0,DFM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DFM,M,W,67.0,D
132854,99291741,1,1,3,1,1,214282,14

In [17]:
df_dict['Dsum'] = df_dict['Dsum'][df_dict['Dsum']['SCREEN_ADA_CODE'].notna() & df_dict['Dsum']['TRIAL_ADA_CODE'].notna() \
                                  & df_dict['Dsum']['TRIAL_DISP_CODE'].notna()]
df_dict['Dsum'] = df_dict['Dsum'].drop_duplicates()
data_merged = pd.merge(data_merged, df_dict['Dsum'], on=['SYS_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132855,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,S,W2600,NaN,N,Y,NaN,19791015.0,5.0,6.0,150.0,M,N,MED,BRO,BLK,GEN,NaN,3.0,1,410654,N,40:967 (C) (2),NaN,NaN,19991105.0,Y,A40P,1.0,ALOG,MGH,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),A40P,199

In [18]:
df_dict['Screen_Ada'] = df_dict['Ada']
df_dict['Screen_Ada'].columns = ['SCREEN_ADA_CODE', 'ADA_NAME_SCREEN_ADA', 'DOB_SCREEN_ADA', \
                                 'RACE_SCREEN_ADA', 'SEX_SCREEN_ADA', 'PARTY_SCREEN_ADA']
data_merged = pd.merge(data_merged, df_dict['Ada'], left_on='SCREEN_ADA_CODE', right_on='SCREEN_ADA_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132855,99306215,1,1,2,1,1,362712,40:967 (C) (2),19991116,50,DFM,N,0,0,0,SIN,Y,3,0,0,0.0,0.0,0.0,0.0,0,N,N

In [19]:
df_dict['Trial_Ada'] = df_dict['Ada']
df_dict['Trial_Ada'].columns = ['TRIAL_ADA_CODE', 'ADA_NAME_TRIAL_ADA', 'DOB_TRIAL_ADA', \
                                'RACE_TRIAL_ADA', 'SEX_TRIAL_ADA', 'PARTY_TRIAL_ADA']
data_merged = pd.merge(data_merged, df_dict['Ada'], on='TRIAL_ADA_CODE', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [20]:
data_merged = pd.merge(data_merged, df_dict['Event'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'CHRG_SEQ_NBR', 
                                                          'DISP_SEQ_NBR', 'SENT_SEQ_NBR', 'BOND_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged.drop(columns=['ADA_CODE_EVENT'], inplace=True)
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [21]:
data_merged = pd.merge(data_merged, df_dict['Parm'], left_on='EVENT_CODE_EVENT', right_on='EVENT_CODE_PARM', how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [22]:
data_merged = pd.merge(data_merged, df_dict['Wdxr'], on=['SYS_NBR', 'DFDN_SEQ_NBR', 'AREG_SEQ_NBR', 'BOFI_NBR'], how='left')
data_merged

,SYS_NBR,DFDN_SEQ_NBR,AREG_SEQ_NBR,CHRG_SEQ_NBR,DISP_SEQ_NBR,SENT_SEQ_NBR,BOFI_NBR,CHARGE_CODE_SENT,DISP_DATE_SENT,DISP_CODE_SENT,JUDGE_CODE_SENT,APPEAL_FLAG,CONFINEMENT_YEARS,CONFINEMENT_MONTHS,CONFINEMENT_DAYS,SENTENCE_TYPE,HABITUAL_OFFENDER_FLAG_SENT,ACTIVE_PROB_YEARS,ACTIVE_PROB_MONTHS,ACTIVE_PROB_DAYS,FINE_AMOUNT,VICTIM_FUND_AMOUNT,RESTITUTION_AMOUNT,INDIGENT_DEFENDER_FUND_AMOUNT,JUDICIAL_FUND_AMOUNT,FINE_SUSPENDED_FLAG,COMMUNITY_SERVICE_FLAG,CONDITION_OF_PROBATION_FLAG,CREDIT_TIME_SERVED_FLAG,SENTENCE_LOCATION,SUSPENDED_SENTENCE_FLAG,PROBATION_TYPE,PHONIC_KEY,CUSTODY_CODE,JUVENILE_FLAG_x,CRIMINAL_FLAG,FBI_NBR,DOB_DFDN,HEIGHT_FEET,HEIGHT_INCHES,WEIGHT,SEX_DFDN,RACE_DFDN,SKIN_COLOR,EYE_COLOR,HAIR_COLOR,CASE_TYPE,CASE_TYPE_2,CASE_CLASS,NBR_OF_DFDN,CASE_NBR,JUVENILE_FLAG_y,LEAD_CHARGE_CODE,CASE_CAT_1,CASE_CAT_2,CHARGE_DATE,LEAD_CHARGE_FLAG,SUB_CHARGE_CODE,COUNTS,CHRG_ADA_CODE,CHRG_JUDGE_CODE,BOND_MADE_AMOUNT,BOND_SET_AMOUNT,BOND_MADE_FLAG,CHARGE_CODE,CHARGE_CAT,CHARGE_CLASS,CHARGE_CODE_DISP,SUB_CHARGE_CODE_DISP,DISP_DATE_DISP,DISP_CODE_DISP,JUDGE_CODE_DISP,LESSER_PLEA_CHARGE_CODE_DISP,PRCN_SEQ_NBR,CONDITION_CODE,CONDITION_YEARS,CONDITION_MONTHS,CONDITION_DAYS,CONDITION_HOURS,BOND_SEQ_NBR,BOND_STATUS_CODE,BOND_TYPE_CODE,BDSR_SEQ_NBR,BOND_TYPE,RELEASED_DATE,JUDGE_CODE,SEX_JUDGE,RACE_JUDGE,AGE_JUDGE,PARTY_JUDGE,SCREEN_ADA_CODE,TRIAL_ADA_CODE,TRIAL_DISP_CODE,ADA_NAME_SCREEN_ADA,DOB_SCREEN_ADA,RACE_SCREEN_ADA,SEX_SCREEN_ADA,PARTY_SCREEN_ADA,ADA_NAME_TRIAL_ADA,DOB_TRIAL_ADA,RACE_TRIAL_ADA,SEX_TRIAL_ADA,PARTY_TRIAL_ADA,EVENT_CODE_EVENT,EVENT_CODE_PARM,BOND_EVENT_FLAG,WHO_CODE,WITN_TYPE,WITN_IS_VICTIM_FLAG
0,84000280,1,1,1,1,1,246712,40:967 (C) (2),19850222,50,NaN,N,2,0,0,SIN,N,1,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,S,R1200,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,306212,N,40:967 (C) (2),NaN,NaN,19850105.0,Y,ERB,1.0,JCHA,NaN,0.0,0.0,N,40:967 (C) (2),NAR,3,40:967 (C) (2),ERB,19850222.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84000405,1,1,1,1,1,197485,0.640277778,19911125,50,NaN,N,0,0,78,SIN,N,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,S5125,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1,302074,N,14:82,NaN,NaN,19840611.0,Y,NaN,1.0,ROBA,NaN,5000.0,5000.0,Y,NaN,NaN,NaN,14:82,NaN,19911125.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84000429,1,1,1,1,1,142503,40:966 (D) (2),19830930,50,NaN,N,1,0,0,SIN,N,2,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,Y,U,G3000,NaN,N,N,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NSP,NaN,3.0,1,297291,N,40:966 (D) (2),NaN,NaN,19830705.0,Y,DFM,1.0,CBEL,NaN,2000.0,2000.0,Y,40:966 (D) (2),NAR,3,40:966 (D) (2),DFM,19830930.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CMM,1.0,CMM,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84000873,2,1,1,1,1,238381,0.645138889,19860602,50,NaN,N,0,18,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,19840926.0,Y,CJW,1.0,RMOS,NaN,100.0,100.0,Y,NaN,NaN,NaN,14:89,CJW,19860602.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,RM,CBP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,84000873,2,1,2,1,1,238381,23:49.1,19860730,30,NaN,N,0,30,0,SIN,Y,0,0,0,0.0,0.0,0.0,0.0,0,N,N,N,N,NaN,N,NaN,M2354,NaN,N,N,583958L8,19560815.0,0.0,0.0,0.0,F,B,NaN,NaN,NaN,VIC,NaN,3.0,2,303852,N,14:89 (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15:529.1,CJW,19860730.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [23]:
for name in data_merged.columns:
    if name.endswith('_x'):
        print(name)

JUVENILE_FLAG_x
